In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import pyswarms as ps
import numpy as np
import pandas as pd
import os

In [2]:
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
# Define o tamanho da imagem e os tipos de rua
im_size = 350  # Exemplo de tamanho de imagem
street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'

In [4]:
# Criando dados de treino, validação e teste
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

# Contando as ocorrências de cada tipo de rua e imprimindo
streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: recycle    1675
clean      1625
litter     1505
dtype: int64


In [5]:
# Convertendo etiquetas para valores numéricos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Embaralhando e dividindo os dados de treino e validação
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=415)

# Imprimindo as formas dos conjuntos de dados
print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (3844, 350, 350, 3)
Validation shape: (961, 350, 350, 3)
Test shape: (600, 350, 350, 3)


In [18]:
# Defina as funções de treinamento de rede e fitness para o PSO
def train_network(hyperparameters, train_x, train_y, val_x, val_y):
    """learning_rate, dropout_rate = hyperparameters
    # Definindo o modelo da rede neural convolucional
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        Dense(len(street_types), activation='softmax')   
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_x, 
              train_y, 
              epochs=5, 
              batch_size=64,
              validation_data=(val_x, val_y), 
              verbose=0)
    
    validation_loss, validation_accuracy = model.evaluate(val_x, val_y, verbose=0)"""
    validation_loss = 0.1
    
    return validation_loss

In [19]:
def fitness_function(x, train_x, train_y, val_x, val_y):
    n_particles = x.shape[0]
    losses = []
    for i in range(n_particles):
        # x[i] contém os hiperparâmetros para a i-ésima partícula
        hyperparameters = x[i]
        loss = train_network(hyperparameters, train_x, train_y, val_x, val_y)
        losses.append(loss)
    return np.array(losses)

In [20]:
# Inicializando o otimizador GlobalBestPSO do pyswarms
bounds = [(0.0001, 0.1), (0.0, 0.5)]  # (min_learning_rate, max_learning_rate), (min_dropout_rate, max_dropout_rate)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(n_particles=5, 
                                    dimensions=2, 
                                    options=options, 
                                    bounds=bounds)

# Executando o PSO para encontrar os melhores hiperparâmetros
cost, best_pos = optimizer.optimize(fitness_function, 
                                    iters=5, 
                                    train_x=train_x, 
                                    train_y=train_y, 
                                    val_x=val_x, 
                                    val_y=val_y)

best_learning_rate, best_dropout_rate = best_pos

2023-11-07 16:58:39,724 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|5/5, best_cost=0.1
2023-11-07 16:58:39,755 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1, best pos: [7.15676915e-05 3.96627299e-01]


In [14]:
final_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(best_dropout_rate),
    Dense(len(street_types), activation='softmax')   
])

final_model.compile(optimizer=Adam(learning_rate=best_learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

history = final_model.fit(train_x, 
                          train_y, 
                          epochs=5, 
                          batch_size=32, 
                          validation_data=(test_x, test_y))

NameError: name 'test_x' is not defined

In [ ]:
# Avaliação do modelo nos dados de teste
test_loss, test_accuracy = final_model.evaluate(test_x, test_y, verbose=1)
print(f"Test accuracy: {test_accuracy*100:.2f}%")

# Salvando o modelo treinado
final_model.save('final_model.h5')

# Previsões no conjunto de teste
y_pred_test = final_model.predict(test_x)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_true = np.argmax(test_y, axis=1)  # Supondo que 'test_y' é one-hot encoded

# Matriz de confusão
confusion_mtx = confusion_matrix(y_true, y_pred_test_classes)
print("Confusion Matrix:")
print(confusion_mtx)

# Relatório de classificação
class_report = classification_report(y_true, y_pred_test_classes, target_names=label_encoder.classes_)
print("Classification Report:")
print(class_report)

# Cálculo de AUC para cada classe
y_probs = final_model.predict(test_x)  # Supondo que existam várias classes
for i, class_name in enumerate(label_encoder.classes_):
    auc_score = roc_auc_score(test_y[:, i], y_probs[:, i])
    print(f"AUC for {class_name}: {auc_score:.2f}")